In [1]:
import ccxt.async_support as ccxt
import time
import pandas as pd
import numpy as np
import warnings
from IPython.display import clear_output
from matplotlib import pyplot as plt
import json

In [2]:
def calc_vol(part):
    if part.coin.values[0].split('/')[1]=='USDT':
        vol = np.min(part[['bid_vol','ask_vol']].values)
    elif part.coin.values[0].split('/')[1]=='BTC':
        vol = np.min(part[['bid_vol','ask_vol']].values) *trading_df.loc[trading_df.coin=='BTC/USDT','ask'].values[0]
    elif part.coin.values[0].split('/')[1]=='ETH':
        vol = np.min(part[['bid_vol','ask_vol']].values) *trading_df.loc[trading_df.coin=='ETH/USDT','ask'].values[0]
    return vol

def calc_price(orderbook):
    price=np.array([])
    vol=np.array([])
    index=0
    while np.sum(vol*price)<100 and index < len(orderbook)-1:
        price = np.append(price, orderbook[index][0])
        vol = np.append(vol, orderbook[index][1])
        index+=1
    value = np.sum(price*vol)/np.sum(vol)
    return value
def calc_spread(orderbook):
    spread = (orderbook['asks'][0][0]/orderbook['bids'][0][0] - 1)*100
    return spread

binance = ccxt.binance()
ftx = ccxt.ftx()
gateio = ccxt.gateio()
kraken = ccxt.kraken()
gemini = ccxt.gemini()
poloniex = ccxt.poloniex()   

In [3]:

#[binance, ftx, gateio, kraken, gemini, poloniex]
coins={}
for market in [binance, ftx]:
    markets = await market.load_markets()
    for coin in markets.keys():
        if coin not in coins:
            coins[coin]=[market]
        else:
            coins[coin].append(market)
good_coins = []
for coin in coins.keys():
    if len(coins[coin])>=2:
        good_coins.append(coin)
        
trading_df = pd.DataFrame([])
trading_df['coin']= good_coins
trading_df[['bid','ask']] = [0, np.inf]
trading_df[['bid_vol','ask_vol','coef','bid_ex','ask_ex','spread']] = 0

to_drop=[]
for coin in trading_df.coin.values:
    if coin.split('/')[1] != 'USDT' or 'BULL' in coin.split('/')[0] or 'BEAR' in coin.split('/')[0] \
    or coin=='SUN/USDT':
        to_drop.append(trading_df.loc[trading_df.coin==coin].index[0])
if 'ETC/USDT' in trading_df.coin.values:
    to_drop.append(trading_df.loc[trading_df.coin=='ETC/USDT'].index[0])
if 'GTC/USDT' in trading_df.coin.values:
    to_drop.append(trading_df.loc[trading_df.coin=='GTC/USDT'].index[0])
trading_df.drop(to_drop, inplace=True)

total_vol=pd.DataFrame({'coin':trading_df.coin.values})
for market in [binance, ftx, gateio, gemini, kraken, poloniex]:
    total_vol[market.id]=0
    
fin_result={}
for coin in trading_df.coin.values:
    fin_result[coin]=0

positions =pd.DataFrame({'coin':good_coins})
positions[['bid_ex','ask_ex', 'bid']]=0
positions['ask']=np.inf
positions[['open_price','current_price']]=0


coefs={}
for ticker in trading_df.coin.values:
    coefs[ticker]=[]

In [8]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)
while True:
    for ticker in trading_df.coin.values:
        ##data
        part = trading_df.loc[trading_df.coin==ticker]
        part.loc[:,['bid','bid_vol','ask_vol', 'spread']] = 0
        part.loc[:,'ask']=np.inf
        
        part2= positions.loc[positions.coin==ticker]
        part2.loc[:,['bid','ask']]=[0,np.inf]
        
        for market in coins[ticker]:
            try:
                orderbook = await market.fetch_order_book(ticker)
            except Exception as e:
                continue

                
            if len(orderbook['bids'])!=0:
                if part2.bid_ex.values[0]!=0:
                    if market.id in part2.bid_ex.to_list():
                        part2.loc[:,'bid'] = calc_price(orderbook['bids'])
                    
                    if market.id in part2.ask_ex.to_list():
                        part2.loc[:,'ask'] = calc_price(orderbook['asks'])
                        
                if calc_price(orderbook['bids']) > part.bid.values[0]:
                    part.loc[:,['bid', 'bid_vol']] = np.array([float(calc_price(orderbook['bids'])),
                                                               float(orderbook['bids'][0][1])]).astype(float)
                    part.loc[:,'bid_ex'] = market.id

                if calc_price(orderbook['asks']) < part.ask.values[0]:
                    part.loc[:,['ask', 'ask_vol']] = np.array([calc_price(orderbook['asks']),
                                                               orderbook['asks'][0][1]]).astype(float)
                    part.loc[:,'ask_ex'] = market.id
                if market.id in part.bid_ex.values or market.id in part.ask_ex.values:
                    spread = calc_spread(orderbook)
                    if spread > part.spread.values[0]:
                        part.loc[:, 'spread'] = spread
                 
        part.loc[:,'coef'] = round((part.bid.values[0]/part.ask.values[0]-1)*100, 2) 
        coefs[ticker].append(part['coef'].values[0])
        current_price = round((part2.ask.values[0]/part2.bid.values[0]-1)*100, 2)
        part2.loc[:,'current_price'] = current_price
        ##trades
        ##open
        if part.coef.values[0]>1.5 and part2.bid_ex.values[0]==0:
            vol = 100
            
            part2.loc[:, ['bid_ex','ask_ex','bid', 'ask','open_price','current_price']] = \
            [part.ask_ex.values[0], part.bid_ex.values[0], 0, np.inf, part.coef.values[0],0]
            
            total_vol.loc[total_vol.coin==ticker, part.bid_ex.values[0]] += vol*2
            total_vol.loc[total_vol.coin==ticker, part.ask_ex.values[0]] += vol*2
        ##close
        if current_price <0.3 and \
        part2.bid_ex.values[0]!=0:
            
            fin_result[ticker]+=vol * (part2.open_price.values[0]-0.8-current_price)/100
            part2.loc[:,['bid_ex','ask_ex','bid','ask']]=0
            
        trading_df.loc[trading_df.index==part.index.values[0]] = part.values
        positions.loc[positions.index==part2.index.values[0]] = part2.values
        time.sleep(1)
    clear_output(wait=True)
    print(trading_df.loc[trading_df.coef>0.5,['coin','coef','bid_ex','ask_ex','spread']])
    print('-------------------------')
    print(positions.loc[(positions.bid_ex!=0) | (positions.open_price!=0), \
                       ['coin','bid_ex','ask_ex','open_price','current_price']])
    

Empty DataFrame
Columns: [coin, coef, bid_ex, ask_ex, spread]
Index: []
-------------------------
Empty DataFrame
Columns: [coin, bid_ex, ask_ex, open_price, current_price]
Index: []


CancelledError: 

In [ ]:
for i in fin_result:
    if fin_result[i]!=0:
        print(i,':', fin_result[i])
print('Total income: ',sum(fin_result.values()))

In [ ]:
uf=[]
for i in coefs:
    for a in coefs[i]:
        if a>1:
            print(i)
            uf.append(i)
            break

In [ ]:
#clear coefs
for i in coefs:
    for a in coefs[i]:
        if a<-5:
            coefs[i].pop(coefs[i].index(a))

In [ ]:
plt.figure(figsize=(20,10))
for i in uf:
    plt.plot(coefs[i], label=i)
    plt.legend(loc = 2)

In [ ]:
for i in uf:
    plt.figure(figsize=(20,4))
    plt.plot(coefs[i])
    plt.title(i)
    plt.grid()
    plt.show()

In [ ]:
for i in coefs:
    plt.plot(coefs[i])

In [ ]:
for i in coefs:
    if len(coefs[i])>5500:
        print(i)

In [ ]:
for i in total_vol.drop('coin',axis=1).columns:
    print (i,':', np.sum(total_vol.drop(total_vol.loc[(total_vol.coin == 'ETC/USDT')|(total_vol.coin =='XTZ/USDT')].index)[i]))

In [7]:
# save coefs
json1 = json.dumps(coefs)
with open("coefs.json","w") as f:
    f.write(json1)
    f.close()